# Introduction au langage Prolog

Prolog, pour programmation en logique est un langage créé par A. Colmerauer, en 1972 à Marseille. La programmation logique est un paradigme différent de la programmation impérative ou de la programmation fonctionnelle. On définit des faits élémentaires et des règles de la manière suivante :

```
apprend(eve, mathematiques).
apprend(benjamin, informatique).
apprend(benjamin, physique).
enseigne(alice, physique).
enseigne(pierre, mathematiques).
enseigne(pierre, informatique).

etudiant_de(E,P):-apprend(E,M), enseigne(P,M).
```

Ces informations décrivent l'ensemble des connaissances du programme. On lit ici "Benjamin apprend l'informatique", ou "Alice enseigne la physique". La dernière ligne est une règle : "L'élève E est étudiant du professeur P si E apprend la matière M et que P enseigne M". Le symbole `:-` se lit *si*, et la virgule entre `apprend(E,M)` et `enseigne(P,M)` sinifie *et*. Ensuite l'utilisateur peut effectuer des requêtes comme ceci : 

`?-etudiant_de(E, pierre)`

L'interpréteur Prolog renvoie alors `E = benjamin` et `E = eve`.

Ce qui différencie la programmation logique des formes plus courantes d'informatique impérative, c'est qu'on n'explique pas au programme comment résoudre la requête, on se contente de décrire le problème.

De très nombreuses introductions au langage Prolog peuvent être trouvées sur internet, je n'en cite qu'une seule qui m'est apparue comme convenable pour une première approche : *Simply Logical: Intelligent Reasoning by Example*, P. Flach, 1994 <cite data-cite="Flach">(Flach)</cite>.

# La représentation des programmes Prolog

La représentation des programmes Prolog dans OCaml se fait de la manière suivante :

* `var` est une variable (commençant par une majuscule en Prolog), identifiée par une chaîne de caractères, et un numéro. Le numéro sera utilisé pour renommer facilement les variables, pour éviter des conflits de noms.

* `atom` est une chaîne de caractères représentant les symboles de prédicats : soit des constantes (arité 0 du prédicat), soit les foncteurs (arité strictement positive). L'arité de `etudiant_de` est 2.

* un `term` est soit une variable, soit sous la forme `prédicat(terme1, terme2, ...)`, qui est donc un atome, et une liste de termes.

* une `clause` est de la forme `etudiant_de(E,P):-apprend(E,M), enseigne(P,M).`, un terme à gauche et une liste de termes à droite. `eleve(samuel).` est une clause, il n'y rien à droite.

Ce n'est pas moi qui ai inventé cette structure particulière : je l'ai retrouvée dans un diapo de cours en Haskell sur le langage Prolog écrit par Alan Smaill en 2009 <cite data-cite="Smaill">(Smaill)</cite>.

Une structure proche est utilisée en Lisp par Peter Norvig dans *Paradigms of Artificial Intelligence Programming*, 1992 <cite data-cite="Norvig">(Norvig)</cite>. Une citation qui permet de comprendre la structure d'un programme en Prolog : *”We will build a single uniform data base of clauses, without distinguishing rules from facts. The simplest representation of clauses is as a cons cell holding the head and the body. For facts, the body will be empty.”*

J'explique plus bas le *pourquoi* de cette structure, qui permet de représenter les clauses de Horn.

## L'analyse lexicale : le lexer

Il faut d'abord transformer le programme qui est une chaîne de caractères, en une liste de symboles. Les `token` sont les différents symboles présents dans un programme Prolog simple. Des variables et des atomes, et ces caractères spéciaux : `( | ) | , | . | :-`.

Quelques remarques sur le lexer : j'ai bien repris les idées du livre *Le Langage Caml*, Pierre Weis et Xavier Leroy, 1992 <cite data-cite="WeisLeroy">(WeisLeroy)</cite>. Une grande différence est l'utilisation des List à la place des Stream. L'utilisation des Stream semble plus intelligente, mais ils ont pour des raisons qui m'échappent ils ont presque disparu des dernières versions d'OCaml (le pattern-matching est beaucoup plus difficile). Avec les listes il y a bien plus de copies, mais nous n'avons pas là de véritable problème de performance.

## L'analyse syntaxique : le parser

```
<Caractère>    -> a..z (ou) A..Z (ou) _ (ou) 0..9
<Mot>          -> <Caractère> (ou) <Caractère> <Mot>
<Prédicat>     -> a..z (ou) a..z <Mot>
<Variable>     -> A..Z (ou) A..Z <Mot>
<Programme>    ->  <Clause> (ou) <Clause> <Programme>
<Clause>       ->  <Terme> . (ou) <Terme> :- <ListeTermes> .
<ListeTermes>  ->  <Terme> (ou) <Terme> , <ListeTermes>
<Terme>        ->  <Variable> (ou) <Predicat> (ou) <Predicat> (<ListeTermes>) (ou) <Tableau>
<Tableau>      ->  [] (ou) [<ListeTermes>] (ou) [<ListeTermes>|<Tableau>] (ou) [<ListeTermes>|<Variable>]
```



Il faut maintenant transformer la liste de `tokens` en une structure formée des types `clause`, `term`, `atom` et `var`.

* Isoler les clauses séparées par des points

* Isoler les termes de droite et de gauche d'une clause séparés par `:-`

* Isoler les différents termes d'une suite de termes séparés par des virgules

* Transformer la liste de symboles en un terme

* Transformer les symboles en clauses, le programme en liste de clauses

Je l'ai écrit moi-même, sans reprendre une structure existante... C'est très inefficace : le programme est parcouru au moins 3 fois par les 3 fonctions d'isolement, avant même d'entrer dans la fonctions récursive `tokenlist_to_term` qui sert réellement à créer l'arbre... De plus, le programme est copié plein de fois dans les fonctions de parcours. Néanmoins le coût total me parait de loin proportionnel à la longueur du code, et probablement proportionnel au niveau d'imbrication `((()))` des parenthèses (ce qui pourrait certainement être évité). Je pense que ce n'est pas le plus important : les coûts d'exécution seront bien plus problématiques...

Je pense qu'il pourrait exister encore des codes Prolog dont la syntaxe est fausse et qui passent le parser. (En tous cas je ne suis pas assuré du contraire). Il faut se demander si l'on peut représenter des programmes faux dans les structures formées des types `clause`, `term`, `atom` et `var`.

Remarque : vous êtes priés de ne rentrer que des codes justes dans le parser, les messages d'erreur sont plus que lacunaires...

# L'algorithme derrière Prolog

Ma référence en ce qui concerne l'algorithme utilisé par Prolog est *Logic Programming and Prolog*, 1990, Nilsson et Małuszyński <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite>. 

Ce qui semble être le premier article traitant du type de résolution utilisé par Prolog est *A Machine-Oriented
Logic Based on the Resolution Principle*, Robinson, 1965 <cite data-cite="Robinson">(Robinson)</cite>. Cet article introduit l'unification, et le *principe de résolution* (Resolution Principle) dont dérive la SLD-resolution (Linear resolution for Definite clauses with Selection function) utilisée par Prolog.

Une *clause définie*, aussi nommée clause de Horn est de la forme `(P1 et P2 et ... et Pn) => Q`. En particulier, `(non P) => Q` n'est pas une clause de Horn. On peut par contre exprimer`(P1 ou P2) => Q` avec des clauses de Horn, il suffit d'écrire les deux clauses `P1 => Q` et `P2 => Q`. Les versions premières de Prolog se limitent aux clauses de Horn, pour une principale raison : la *correction* et la *complétude* (*soundness* and *completeness*) ont été montrée pour la SLD-resolution, qui n'est valide que sur des clauses de Horn. Pour les citations (un peu compliquées) : la SLD-resolution est introduite par Kowalski en 1974, la correction est montrée par Clark en 1979, la complétude a été montrée premièrement par Hill en 1974, mais quelque chose de plus fort a été montré par Clark en 1979. 


## L'unification

L'unification se fait entre deux termes A et B. Deux termes s'unifient s'il existe une substitution `thêta` des variables de A telle que `B = thêta(A)`. 

Soit la clause du programme Prolog `etudiant_de(E,P):-apprend(E,M), enseigne(P,M).`. Si l'on cherche à réaliser la requête `?-étudiant_de(E, pierre)`, on unifie `étudiant_de(E, pierre)` avec `etudiant_de(E,P)`, le membre de gauche de la clause. La substitution thêta remplace `P` par `pierre` et ne modifie pas les autres variables. Pour continuer la recherche, on applique la substitution au termes de droite de la clause.

Voici l'algorithme décrit dans <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite> :
```
E est l'ensemble des équations. Au départ il n'y en a qu'une seule : 
Par exemple E = {etudiant_de(E,P) = étudiant_de(E, pierre)}.

Répéter tant que E change 
(jusqu'à ce que l'on ne puisse plus rien appliquer aux équations)
    Sélectionner une équation  s = t dans E;
    Si s = t est de la forme :
        f(s1, ..., sn) = f(t1, ..., tn) avec n >= 0 
            Alors remplacer l'équation par s1=t1 ... sn=tn
        f(s1, ..., sm) = g(t1, ..., tn) avec f != g 
            Alors ÉCHEC
        X = X 
            Alors supprimer l'équation
        t = X où t n'est pas une variable 
            Alors remplacer l'équation par X = t
        X = t où X != t et X apparait plus d'une fois dans E 
            Si X est un sous-terme de t Alors ÉCHEC
            Sinon on remplace toutes les autres occurences de X par t
```

Il est prouvé que cet algorithme termine et renvoie soit échec, soit un ensemble équivalent des équations sous *forme résolue*. Un ensemble d'équations `{X1 = t1, ..., Xn = tn}` est dit sous forme résolue si les `(Xn)` sont des variables, les `(tn)` sont des termes et aucune des variables `Xn` n'apparait dans les `tn`. Cela permet ensuite de déterminer un unifieur.

Remarques : 

* C'est dommage que l'algorithme se prête si bien à la programmation impérative quand on programme dans un langage fonctionnel. 

* On trouve des algorithmes pour déterminer un unifieur qui ne manipulent pas exactement comme ça des systèmes d'équations, (et qui sont dans des styles plus fonctionnels), mais cet algorithme est le seul que j'arrive à comprendre convenablement, et en plus <cite data-cite="NilssonMaluszynski">(NilssonMaluszynski)</cite> prouve sa terminaison et sa correction, ce qui est très bon point.

* Le test `Si X est un sous-terme de t Alors ÉCHEC` est en pratique pas réalisé dans la plupart des implémentations Prolog (pas comme ça en tout cas), il est très lent et le cas n'arrive que très peu en pratique. On peut donc avoir des boucles infinies... Les versions "modernes" gèrent les unifications de structures infinies... Une citation de <cite data-cite="Norvig">(Norvig)</cite> "*This represents a circular, infinite unification. Some versions of Prolog, notably Prolog II (Giannesini et al. 1986), provide an interpretation for such structures, but it is tricky to define the semantics of infinite structures.*"

À la place de manipuler des ensembles (`Set` existe en OCaml), j'ai juste utilisé des liste triées. La fonction `compare` est magique, elle définit une relation d'ordre pour n'importe quel type. J'ai ici utilisé la version complète de l'algorithme, qui n'est certainement pas la plus efficace. Comme l'a dit Donald Knuth : "*Premature optimization is the root of all evil (or at least most of it) in programming.*"

On définit une substitution comme une liste de couples (var,term), le terme est inséré à la place de la variable.

Si l'on sait que `etudiant_de(E1,P):-apprend(E1,M), enseigne(P,M).` (c) et que l'on veut montrer `étudiant_de(E0, pierre).` (r), alors il faut trouver une substitution qui unifie la tête de la clause (c) avec la requête (r).

Pour l'exemple plus haut, il faut la substitution `thêta = { P/pierre, E1/E0}`. Il faut comprendre "La variable P est remplacée par 'pierre', la variable E1 est remplacée par la variable E0. C'est bien un unifieur, si on l'applique sur la requête et sur la tête de la clause, les deux deviennent égales.

Il peut exister plusieurs unifieurs, par exemple `thêta2 = { P/pierre, E0/samuel, E1/samuel}` en est aussi un. On voit que thêta2, c'est thêta composé avec `{E0/samuel}`, on dit alors que thêta est plus général que thêta2. Nous, nous cherchons le plus général de ces unifieurs, le MGU pour *Most General Unifier*.

Un détail : on peut avoir thêta plus général que oméga, et oméga plus général que théta, la relation n'est donc pas antisymétrique. En fait, le MGU est unique au renommage des variables près.

On peut déduire facilement un MGU du système d'équations sous forme résolue de la fonction `solve` (c'est à ça qu'elle sert). Si `{X1 = t1, ..., Xn = tn}` est sous forme résolue, alors `{X1/t1, ..., Xn/tn}` est un MGU.

Lorsque l'on veut unifier `etudiant_de(E,P)` et `etudiant_de(E,pierre)`, les deux variables `E` ne doivent pas être nommées de la même manière.

Quelques explications sur les derniers deux exemples :

* Je sais que pour tout `Z`, `f(g(Z),Z)` est vraie, et j'aimerais savoir s'il existe des couples `(X,Y)` tels que `f(X,g(Y))` soit vraie. Le programme me répond oui, il suffit de choisir `X = g(g(Y))`

* Je sais que pour tout `(X,Y)`, `f(X,g(Y))` est vraie, et j'aimerais savoir s'il existe des `Z` tels que `f(g(Z),Z)` soit vraie. Le programme me répond oui, il suffit de choisir `Z = g(Y)`

## Le backtracking

Il me semble que nous nous rapprochons du but ! L'unification est une partie très importante de la SLD-resolution. L'autre point important est le backtracking. Encore quelques fonctions pour appliquer des substitutions sur des termes, listes de termes, pour composer des substitutions, pour vérifier que un terme et une clause n'ont pas de variables en commun, pour effectuer les renommages si nécessaire, et après je pense qu'il sera possible d'implémenter le backtracking.

Voilà la manière dont j'ai compris l'algorithme :

* On cherche à satisfaire une requête `<- A1,...An`.

* On a `A1` qui unifie avec `Hi`, où `Hi <- Ci_1,..Ci_m` est une la i-ème clause du programme. On a auparavant renommé toutes les variables de `Hi <- Ci_1,..Ci_m` qui étaient présentes dans `A1`, sinon on ne peut pas appeler `mgu`. On a donc un MGU `thêta1`.

* La nouvelle requête à satisfaire est `<- thêta1(Ci_1,...Ci_m, A2,...An)`.

* On récure, si on a à montrer `thêta_k( ... thêta2( thêta1( _vide_) ) ...)` alors c'est gagné (`_vide_` signifie toujours vrai), la composée `thêta_tot` des `thêta_k` est une substitution des variables de la requête de départ. Ce qu'on veut renvoyer à l'utilisateur c'est l'image des variables de `A1,...An` par `thêta_tot`.

* Si `A1` ne s'unifie avec aucun `Hi`, ça ne sert à rien d'essayer d'unifier `A2`, puisque de toute façon on garde `A1` dans la nouvelle requête. Alors il faut remonter. C'est à dire qu'il faut essayer les autres unifications à l'étape d'avant.

Comment renommer les clauses pour avoir des noms libres à chaque unification ? On peut utiliser le niveau de récursion comme identifiant, que l'on place dans l'entier transporté avec la variable.

* On veut des variables numérotées à 0 dans la requête. 

* On passe 1 lors du premier appel de sld : la première clause utilisée est renommée à 1, les variables de la requête sont toutes à 0.

* La deuxième clause utilisée est renommée à 2, dans la requête il y a des variables à 1 et à 0 : pas de conflit ! etc...

## Que faire maintenant ?

* Beaucoup de tests pour vérifier si tout fonctionne.

* Peut-être nettoyer un peu le code...

* Écrire quelques fonctions comme une ligne de commande interactive, pour une utilisation plus facile. Rendre les résultats plus clairs.

* Proposer une deuxième version de la recherche, qui renvoie tous les résultats possibles, ou qui propose de chercher le résultat suivant.

* Je pensais essayer d'écrire un programme pour "*résoudre*" le Cluedo. J'avais déjà essayé sans succès en C++. Le problème semble plus adapté au langage Prolog : "*tel joueur possède telle carte*", "*si un joueur montre une carte à un autre pour réfuter une supposition (triplet de cartes), c'est qu'il possède une des trois cartes*", sont des règles que l'on peut écrire en Prolog. Malheureusement j'ai trouvé un super mémoire de Master <cite data-cite="Aartun">(Aartun)</cite> qui traite bien le sujet.

* Je regarde un peu comment est-ce que l'on peut sortir des clauses de Horn, mais c'est un sujet compliqué.

* Je voulais voir s'il est possible d'étendre à des langages fonctionels des éléments de programmation logique. C'est aussi un thème intéressant, qui se nomme *functional logic programming*. On trouve plusieurs petits langages sur internet qui utilisent des approches différentes, mais aucun d'eux ne semble bien abouti. Ce serait presque un idéal d'associer des mécanismes efficaces des langages fonctionnels comme l'évaluation paresseuse, et les capacités de la programmation logique pour décrire les problèmes...

* Pour compiler Prolog, il faut se tourner vers les *Machines abstraites de Warren*.